# Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

In [ ]:
%matplotlib inline
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite

# from itertools import permutations

### Data

In [ ]:
# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])

In [ ]:
# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

In [ ]:
!find .. | grep -i employee_movie

In [ ]:
!head ./_data/Employee_Movie_Choices.txt

In [ ]:
G = nx.read_edgelist('./_data/Employee_Movie_Choices.txt', delimiter='\t')
print(nx.info(G))

In [ ]:
list(G.edges(data=True))[:5]

In [ ]:
list(G.nodes(data=True))[:5]

### Bipartite graph

Add nodes attributes named `'type'='movie'` `'type'='employee'`

In [ ]:
G_df = pd.read_csv('./_data/Employee_Movie_Choices.txt', delimiter='\t', skiprows=1, names=['employee', 'movie'])
G_df.head()

In [ ]:
_ = [G.add_node(G_df.loc[i, 'employee'], type='employee') for i, x in enumerate(G_df.index)]
_ = [G.add_node(G_df.loc[i, 'movie'], type='movie') for i, x in enumerate(G_df.index)]
G.nodes(data=True)

### Weighted projection of the graph

Which tells us how many movies different pairs of employees have in common.

In [ ]:
L = [x for x in G if G.node[x]['type']=='employee']
R = [x for x in G if G.node[x]['type']=='movie']
L, R

In [ ]:
B = nx.Graph() 
B.add_nodes_from(L, bipartite=0)
B.add_nodes_from(R, bipartite=1)

B.add_edges_from(G.edges())
assert bipartite.is_bipartite(B) # Check if B is bipartite

#### Get weighted edges

In [ ]:
P_employee.edges(data=True)

In [ ]:
P_employee.get_edge_data('Andy', 'Georgia')

In [ ]:
P_movie.edges(data=True)

In [ ]:
P_movie.get_edge_data('Anaconda', 'Snakes on a Plane')

In [ ]:
employee_weights = [attr['weight'] for u, v, attr in P_employee.edges(data=True)]
employee_weights

In [ ]:
movie_weights = [attr['weight'] for u, v, attr in P_movie.edges(data=True)]
movie_weights

#### Visualise graph

In [ ]:
P_employee = bipartite.weighted_projected_graph(B, L)
nx.draw_networkx(P_employee, width=employee_weights)

In [ ]:
P_movie = bipartite.weighted_projected_graph(B, R)
nx.draw_networkx(P_movie, width=movie_weights)

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [ ]:
# 

G_df = pd.read_csv('./_data/Employee_Relationships.txt', delimiter='\t', skiprows=1, names=['emp1', 'emp2', 'score'])

G1 = nx.from_pandas_dataframe(G_df, 'emp1', 'emp2', edge_attr='score')
G2 = P_employee
G3 = nx.compose(G1, G2)
print(G3.edges(data=True))

In [ ]:
df = pd.DataFrame(list(G3.edges(data=True)), columns=['emp1', 'emp2', 'score_weight'])
df.head()

In [ ]:
def extract_dict_value(x, ftr):
    try: return x[ftr]
    except: return 0

df['score'] = df['score_weight'].map(lambda x: extract_dict_value(x, 'score'))
df['weight'] = df['score_weight'].map(lambda x: extract_dict_value(x, 'weight'))

In [ ]:
df.head()

#### Correlation between weight and score

Do employees that have a high relationship score also like the same types of movies

In [ ]:
cor = df['weight'].corr(df['score'])
cor